# Yield calculation: drive ag model with climate model outputs

In [1]:
import numpy as np
import pandas as pd

### Regression coefficients

In [2]:
# Get coefficients
coeff = pd.read_csv('../fit_model/ag_model_coeff_final.csv')
coeff['GEOID'] = coeff['GEOID'].astype(str).str.zfill(5)

In [3]:
coeff.head()

,GEOID,GDDc,KDDc,Pc,P2c,R2,pval
0,01001,0.000650,-0.009133,0.638664,-0.306875,0.452173,7.862108e-05
1,01003,0.000000,-0.002219,1.515195,-0.644551,0.199122,7.038708e-02
2,01005,0.000148,-0.004832,0.820174,-0.524230,0.337459,2.971402e-03
3,01009,0.000977,-0.010829,0.000000,-0.538866,0.631368,7.569497e-08
4,01011,0.000290,-0.006119,0.000000,-0.099910,0.294675,2.883050e-02


## GMFD

In [5]:
# Import GMFD data
gmfd = pd.read_csv('../../agvars/GMFD/agvar_historical_gmfd.csv')
gmfd = gmfd.query('Year >= 1960 and Year <= 2005')
gmfd.sort_values(by = ['GEOID', 'Year'], inplace = True)
gmfd['GEOID'] = gmfd['GEOID'].astype(str).str.zfill(5)
gmfd['prcp2'] = gmfd['prcp']**2
gmfd.set_index(['GEOID', 'Year'], inplace = True)

In [6]:
# Subtract means
for agvar in gmfd.columns:
    gmfd[agvar] = gmfd[agvar] - gmfd[agvar].mean(level = 'GEOID')

In [7]:
gmfd

gdd       egdd      prcp     prcp2
GEOID Year                                           
01001 1960  -60.223147  13.023867 -0.022750 -0.062252
      1961 -127.704408 -32.404401  0.040302  0.029222
      1962   34.688328  54.663959 -0.111085 -0.177034
      1963  103.941708  16.534303 -0.074131 -0.130915
      1964   18.741261  -6.690650  0.179908  0.260053
...                ...        ...       ...       ...
56045 2001  100.391783  22.470054 -0.022827 -0.016458
      2002   83.175751  35.856844 -0.065962 -0.037130
      2003   62.098236  31.335008 -0.044619 -0.027366
      2004  -31.307908 -13.726102 -0.084411 -0.044836
      2005   47.132736  18.118639  0.047100  0.024961

[142968 rows x 4 columns]

In [8]:
# Calculate Yields
gmfd = pd.merge(coeff, gmfd.reset_index(), on = ['GEOID'], how = 'inner')
gmfd.set_index(['GEOID', 'Year'], inplace = True)
gmfd['GMFD'] = gmfd['gdd']*gmfd['GDDc'] + gmfd['egdd']*gmfd['KDDc'] + gmfd['prcp']*gmfd['Pc'] + gmfd['prcp2']*gmfd['P2c']
gmfd = gmfd.filter(['GEOID', 'Year', 'GMFD'])

In [9]:
# Save
gmfd.to_csv('./output/GMFD/res_yield_60-05_gmfd.csv')

## NEX

In [4]:
nex_hist = ["agvar_historical_r1i1p1_ACCESS1-0.csv",
"agvar_historical_r1i1p1_BNU-ESM.csv",
"agvar_historical_r1i1p1_CCSM4.csv",
"agvar_historical_r1i1p1_CESM1-BGC.csv",
"agvar_historical_r1i1p1_CNRM-CM5.csv",
"agvar_historical_r1i1p1_CSIRO-Mk3-6-0.csv",
"agvar_historical_r1i1p1_CanESM2.csv",
"agvar_historical_r1i1p1_GFDL-CM3.csv",
"agvar_historical_r1i1p1_GFDL-ESM2G.csv",
"agvar_historical_r1i1p1_GFDL-ESM2M.csv",
"agvar_historical_r1i1p1_IPSL-CM5A-LR.csv",
"agvar_historical_r1i1p1_IPSL-CM5A-MR.csv",
"agvar_historical_r1i1p1_MIROC-ESM-CHEM.csv",
"agvar_historical_r1i1p1_MIROC-ESM.csv",
"agvar_historical_r1i1p1_MIROC5.csv",
"agvar_historical_r1i1p1_MPI-ESM-LR.csv",
"agvar_historical_r1i1p1_MPI-ESM-MR.csv",
"agvar_historical_r1i1p1_MRI-CGCM3.csv",
"agvar_historical_r1i1p1_NorESM1-M.csv",
"agvar_historical_r1i1p1_bcc-csm1-1.csv",
"agvar_historical_r1i1p1_inmcm4.csv"]

nex_proj = ["agvar_rcp85_r1i1p1_ACCESS1-0.csv",
"agvar_rcp85_r1i1p1_BNU-ESM.csv",
"agvar_rcp85_r1i1p1_CCSM4.csv",
"agvar_rcp85_r1i1p1_CESM1-BGC.csv",
"agvar_rcp85_r1i1p1_CNRM-CM5.csv",
"agvar_rcp85_r1i1p1_CSIRO-Mk3-6-0.csv",
"agvar_rcp85_r1i1p1_CanESM2.csv",
"agvar_rcp85_r1i1p1_GFDL-CM3.csv",
"agvar_rcp85_r1i1p1_GFDL-ESM2G.csv",
"agvar_rcp85_r1i1p1_GFDL-ESM2M.csv",
"agvar_rcp85_r1i1p1_IPSL-CM5A-LR.csv",
"agvar_rcp85_r1i1p1_IPSL-CM5A-MR.csv",
"agvar_rcp85_r1i1p1_MIROC-ESM-CHEM.csv",
"agvar_rcp85_r1i1p1_MIROC-ESM.csv",
"agvar_rcp85_r1i1p1_MIROC5.csv",
"agvar_rcp85_r1i1p1_MPI-ESM-LR.csv",
"agvar_rcp85_r1i1p1_MPI-ESM-MR.csv",
"agvar_rcp85_r1i1p1_MRI-CGCM3.csv",
"agvar_rcp85_r1i1p1_NorESM1-M.csv",
"agvar_rcp85_r1i1p1_bcc-csm1-1.csv",
"agvar_rcp85_r1i1p1_inmcm4.csv"]

In [8]:
# Run calculation
def run_calc(name, hist=True, mid=False, end=False):
    # Read in data
    data = pd.read_csv("../../agvars/NEX-GDDP/" + name)
    data["GEOID"] = data["GEOID"].astype(str).str.zfill(5)
    data.set_index(['GEOID', 'Year'], inplace=True)
    
    # Select time period
    if hist:
        data = data.query('Year >= 1960 and Year <= 2005')
    elif mid:
        data = data.query('Year >= 2040 and Year <= 2059')
    elif end:
        data = data.query('Year >= 2080 and Year <= 2099')
    
    # Square precip
    data['prcp2'] = data['prcp']**2
    
    # Subtract means
    for agvar in data.columns:
        data[agvar] = data[agvar] - data[agvar].mean(level = 'GEOID')

    # Calculate yields
    data = pd.merge(coeff, data.reset_index(), on = 'GEOID', how = 'outer')
    data['yield'] = data['gdd']*data['GDDc'] + data['egdd']*data['KDDc'] + data['prcp']*data['Pc'] + data['prcp2']*data['P2c']
    data = data.filter(['GEOID', 'Year', 'yield'])
    
    # Save
    timeID = hist*'60-05' + mid*'40-59' + end*'80-99'
    data.to_csv("./output/NEX-GDDP/res_" + timeID + '_yield_' + name[6:], index=False)

In [19]:
for model in nex_hist:
    run_calc(model)

/Users/davidlafferty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/davidlafferty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [7]:
for model in nex_proj:
    run_calc(model, mid=True, hist=False)

/Users/davidlafferty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/davidlafferty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
for model in nex_proj:
    run_calc(model, end=True, hist=False)

/Users/davidlafferty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/davidlafferty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## CMIP

In [12]:
cmip_all = ["agvar_ACCESS1-0.historical+rcp85.csv",
"agvar_BNU-ESM.historical+rcp85.csv",
"agvar_CCSM4_historical+rcp85.csv",
"agvar_CESM1-BGC.historical+rcp85.csv",
"agvar_CNRM-CM5.historical+rcp85.csv",
"agvar_CSIRO-Mk3-6-0.historical+rcp85.csv",
"agvar_CanESM2.historical+rcp85.csv",
"agvar_GFDL-CM3.historical+rcp85.csv",
"agvar_GFDL-ESM2G.historical+rcp85.csv",
"agvar_GFDL-ESM2M.historical+rcp85.csv",
"agvar_IPSL-CM5A-LR.historical+rcp85.csv",
"agvar_IPSL-CM5A-MR.historical+rcp85.csv",
"agvar_MIROC-ESM-CHEM.historical+rcp85.csv",
"agvar_MIROC-ESM.historical+rcp85.csv",
"agvar_MIROC5.historical+rcp85.csv",
"agvar_MPI-ESM-LR.historical+rcp85.csv",
"agvar_MPI-ESM-MR.historical+rcp85.csv",
"agvar_MRI-CGCM3.historical+rcp85.csv",
"agvar_NorESM1-M.historical+rcp85.csv",
"agvar_bcc-csm1-1_historical+rcp85.csv",
"agvar_inmcm4.historical+rcp85.csv"]

In [11]:
# Run calculation
def run_calc(name, hist=True, mid=False, end=False):
    # Read in data
    data = pd.read_csv("../../agvars/CMIP/" + name)
    data["GEOID"] = data["GEOID"].astype(str).str.zfill(5)
    data.set_index(['GEOID', 'Year'], inplace=True)
    
    # Select time period
    if hist:
        data = data.query('Year >= 1960 and Year <= 2005')
    elif mid:
        data = data.query('Year >= 2040 and Year <= 2059')
    elif end:
        data = data.query('Year >= 2080 and Year <= 2099')
    
    # Square precip
    data['prcp2'] = data['prcp']**2
    
    # Subtract means
    for agvar in data.columns:
        data[agvar] = data[agvar] - data[agvar].mean(level = 'GEOID')

    # Calculate yields
    data = pd.merge(coeff, data.reset_index(), on = 'GEOID', how = 'outer')
    data['yield'] = data['gdd']*data['GDDc'] + data['egdd']*data['KDDc'] + data['prcp']*data['Pc'] + data['prcp2']*data['P2c']
    data = data.filter(['GEOID', 'Year', 'yield'])
    
    # Save
    timeID = hist*'60-05' + mid*'40-59' + end*'80-99'
    data.to_csv("./output/CMIP/res_" + timeID + '_yield_' + name[6:], index=False)

In [22]:
for model in cmip_all:
    run_calc(model)

/Users/davidlafferty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/davidlafferty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [13]:
for model in cmip_all:
    run_calc(model, mid=True, hist=False)

/Users/davidlafferty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/davidlafferty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [14]:
for model in cmip_all:
    run_calc(model, end=True, hist=False)

/Users/davidlafferty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/davidlafferty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
